In [12]:
module NCS
export Particle,init_pop,optimize
const r = 0.99
const λrange= 0.1
const epoch = 100
type Particle{T,N}
    x::Array{T,N} # position
    v::T # value
    vn::T # normalized value
    x′::Array{T,N} # trial position
    v′::T  # trial value
    vn′::T # normalized tial value
    σ::Array{T,N} # diagnol elements of Σ (sampling covariance matrix)
    Corr::T # correlation value
    Corr′::T # tiral correlation value
    flag::T # successful trial in an epoch
    λ::T # λ>0 is a parameter to balance exploration and exploitation.
end
Base.show(p::Particle) = println("x=",p.x," f(x)=",p.v)
function init_pop(f,l,u; N=10)
    g = [Particle(l+(u-l).*rand(size(l)),Inf,Inf,
        l+(u-l).*rand(size(l)),Inf,Inf,
        (u-l)/N,Inf,Inf,0.0,1.0) for i=1:N]
    for p in g
        p.v = f(p.x)
    end
    g
end
function BD(p, q)
    Corr = Corr′ = 0  
    @inbounds for d in eachindex(p.x)
        Δx = p.x[d] - q.x[d]
        Δx′ = p.x′[d] - q.x[d]
        σ2 = p.σ[d]^2; σ2′ = q.σ[d]^2
        c = (σ2+σ2′)/2
        Corr += 1/8*Δx^2/c + 1/2*(log(c) - 0.5*(log(σ2) + log(σ2′)))
        Corr′+= 1/8*Δx′^2/c + 1/2*(log(c) - 0.5*(log(σ2) + log(σ2′)))
    end
    Corr,Corr′
end
function fitness_values{T,N}(f,g::Array{Particle{T,N}})
#     Threads.@threads 
    for p in g
        p.x′ .= p.x .+ p.σ.*randn(size(p.x))
        p.v′ = f(p.x′)
    end
end

function normalize_fitness_values(g)
    v_min = minimum(min(p.v,p.v′) for p in g)
    for p in g
        p.vn = p.v - v_min;
        p.vn′ = p.v′ - v_min;
        p.vn,p.vn′ = p.vn/(p.vn+p.vn′),p.vn′/(p.vn+p.vn′)
    end
end
function correlation_values(g)
    for p in g
        p.Corr = p.Corr′ = Inf
        for q in g
            c,c′ = BD(p,q)
            if c < p.Corr
                p.Corr = c
            end
            if c′ < p.Corr′
                p.Corr′ = c′
            end
        end
    end
end
function normalize_correlation_values(g)    
    for p in g
        p.Corr,p.Corr′ = p.Corr/(p.Corr+p.Corr′+1e-20),
        p.Corr′/(p.Corr+p.Corr′+1e-20)
    end
end
function selection(g)
    for p in g
        if p.λ*p.Corr′ > p.vn′
            p.x = copy(p.x′)
            p.v = p.v′
            p.flag += 1.0
        end
    end
end         

function update_parameters(g,t,T)
    for p in g
        p.λ = 1 + λrange*(1-t/T)*randn()
    end
    if mod(t, epoch) == 0
        for p in g         
            if p.flag/epoch > 0.2
                p.σ /= r;
            elseif p.flag/epoch < 0.2
                p.σ *= r;
            end
            p.flag = 0.0
        end
    end
end
function best(p0,g)
    for p in g
            if p.v < p0.v
            p0.x = copy(p.x)
            p0.v = p.v
        end
    end
end
function optimize(f,g; T=10,disp=false)
    p0 = deepcopy(g[1])
    for t = 0:T
        fitness_values(f,g)
        normalize_fitness_values(g)
        correlation_values(g)
        normalize_correlation_values(g)
        selection(g)
        best(p0,g)
        update_parameters(g,t,T)
        if disp == true && mod(t,100) == 0
            show(p0)
        end
    end
    p0.x,p0.v
end
end

NCS

In [13]:
addprocs(3)

ERROR (unhandled task failure): listen: address already in use (EADDRINUSE)
 in uv_error at ./libuv.jl:68 [inlined]
 in #listen#393(::Int64, ::Function, ::Base.InetAddr{IPv4}) at ./socket.jl:700
 in (::Base.#kw##listen)(::Array{Any,1}, ::Base.#listen, ::Base.InetAddr{IPv4}) at ./<missing>:0
 in #serve#4(::Array{Any,1}, ::Function, ::Int64) at /home/rluser/.julia/v0.5/Atom/src/comm.jl:92
 in serve(::Int64) at /home/rluser/.julia/v0.5/Atom/src/comm.jl:92
 in (::##2#4{Module})() at ./task.jl:360
ERROR (unhandled task failure): listen: address already in use (EADDRINUSE)
 in uv_error at ./libuv.jl:68 [inlined]
 in #listen#393(::Int64, ::Function, ::Base.InetAddr{IPv4}) at ./socket.jl:700
 in (::Base.#kw##listen)(::Array{Any,1}, ::Base.#listen, ::Base.InetAddr{IPv4}) at ./<missing>:0
 in #serve#4(::Array{Any,1}, ::Function, ::Int64) at /home/rluser/.julia/v0.5/Atom/src/comm.jl:92
 in serve(::Int64) at /home/rluser/.julia/v0.5/Atom/src/comm.jl:92
 in (::##2#4{Module})() at ./task.jl:360
ERRO

3-element Array{Int64,1}:
  8
  9
 10

In [16]:
@everywhere begin
import NCS
const A = 10
rastrigin(x) = A*length(x) + sum(x.^2 .- A*cos.(2π*x))
end
g = NCS.init_pop(x->rastrigin(x),-ones(3),ones(3); N=50)
@time x0,y0 = NCS.optimize(x->rastrigin(x),g; T=1,disp=true)

LoadError: On worker 2:
ArgumentError: Module NCS not found in current path.
Run `Pkg.add("NCS")` to install the NCS package.
 in require at ./loading.jl:365
 in eval at ./boot.jl:234
 in #37 at ./multi.jl:1957
 in #625 at ./multi.jl:1421
 in run_work_thunk at ./multi.jl:1001
 in macro expansion at ./multi.jl:1421 [inlined]
 in #624 at ./event.jl:68
 in #remotecall_fetch#606(::Array{Any,1}, ::Function, ::Function, ::Base.Worker) at ./multi.jl:1070
 in remotecall_fetch(::Function, ::Base.Worker) at ./multi.jl:1062
 in #remotecall_fetch#609(::Array{Any,1}, ::Function, ::Function, ::Int64) at ./multi.jl:1080
 in remotecall_fetch(::Function, ::Int64) at ./multi.jl:1080
 in (::##38#40)() at ./multi.jl:1959

...and 8 other exceptions.


In [20]:
import NCS
const A = 10
rastrigin(x) = A*length(x) + sum(x.^2 .- A*cos.(2π*x))
g = NCS.init_pop(x->rastrigin(x),-ones(3),ones(3); N=50)
@time x0,y0 = NCS.optimize(x->rastrigin(x),g; T=1000,disp=true)

x=[0.0518152,-0.125869,-0.188115] f(x)=9.755755874180029


x=[0.00376503,0.0186632,-0.0712909] f(x)=1.0634797496767305
x=[-1.00829,-0.00460273,0.00827538] f(x)=1.047993065094019
x=[-0.0372093,-0.0359278,-0.0174312] f(x)=0.5886644745655047
x=[-0.0372093,-0.0359278,-0.0174312] f(x)=0.5886644745655047
x=[-0.0152281,-0.0253725,-0.0319281] f(x)=0.3749871892100529
x=[0.0257391,-0.029334,0.0173396] f(x)=0.360973247491156
x=[0.0257391,-0.029334,0.0173396] f(x)=0.360973247491156
x=[0.0257391,-0.029334,0.0173396] f(x)=0.360973247491156
x=[-0.0105309,-0.0137315,0.0212872] f(x)=0.14914529929496823
x=[-0.0105309,-0.0137315,0.0212872] f(x)=0.14914529929496823
  3.732736 seconds (61.07 M allocations: 1.014 GB, 2.89% gc time)


([-0.0105309,-0.0137315,0.0212872],0.14914529929496823)

In [ ]:
using ProfileView